# Step 3: Model Building

The final feature engineered dataset is then split into two namely a train and a test dataset based on a date-time stamp. Then two models namely a Random Forest Classifier and Decision Tree Classifier are built on the training dataset and then scored on the test dataset.

In this notebook, we will load the data stored in Azure Blob containers in the previous Feature Engineering notebook (Code/feature_engineering.ipynb). 


In [1]:
## setup our environment by importing required libraries
import os
import csv

import pandas as pd
import io
import requests

import glob
import json
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

# for creating pipelines and model
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, VectorIndexer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import *
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# setup the pyspark environment
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Load data from Azure Blob storage container 

We have previously feature engineering on the dataset. 

We'll load this file from blob, and create our models here. Based on the results we will deploy one of these models in the next notebook. 

Since the Azure Blob storage account name and account key are not passed between notebooks, you'll need to provide those here again.

In [2]:
# define parameters 
ACCOUNT_NAME = "pdmvienna"
ACCOUNT_KEY = "PDuXK61GpmMVWMrWdvr29THbPdlOXa61fN5RfgQV/jBO8berC1zLzZ678Nxrx+D3CRp4+ZvSff9al+lrUh8qUQ=="
CONTAINER_NAME = "featureengineering"

# define your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

In [3]:
# load the previous created final dataset into the workspace
# create a local path where to store the results later
LOCAL_DIRECT = 'model_result.parquet'
if not os.path.exists(LOCAL_DIRECT):
    os.makedirs(LOCAL_DIRECT)
    print('DONE creating a local directory!')

# define your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# download the entire parquet result folder to local path for a new run 
for blob in my_service.list_blobs(CONTAINER_NAME):
    if 'featureengineering_files.parquet' in blob.name:
        local_file = os.path.join(LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

data = spark.read.parquet('model_result.parquet')
#data.persist()
data.show(5)
print('Feature engineering final dataset files loaded!')

DONE creating a local directory!
+---------+--------------------+------------------+--------------------+----------------------+-----------------------+-------------------+---------------------+-----------------------+------------------------+------------------+-------------------+---------------------+----------------------+------------------+--------------------+----------------------+-----------------------+------------------------+------------------------+------------------------+------------------------+------------------------+-----------------+-----------------+-----------------+-----------------+------+---+-------------+--------+-------+
|machineID|        dt_truncated|volt_rollingmean_3|rotate_rollingmean_3|pressure_rollingmean_3|vibration_rollingmean_3|volt_rollingmean_24|rotate_rollingmean_24|pressure_rollingmean_24|vibration_rollingmean_24| volt_rollingstd_3|rotate_rollingstd_3|pressure_rollingstd_3|vibration_rollingstd_3|volt_rollingstd_24|rotate_rollingstd_24|pressure_rol

# Prepare the Training/Testing data

When working with data that comes with time-stamps such as telemetry and errors as in this example, splitting of data into training, validation and test sets should be performed carefully to prevent overestimating the performance of the models. In predictive maintenance, the features are usually generated using laging aggregates and consecutive examples that fall into the same time window may have similar feature values in that window. If a random splitting of training and testing is used, it is possible for some portion of these similar examples that are in the same window to be selected for training and the other portion to leak into the testing data. Also, it is possible for training examples to be ahead of time than validation and testing examples when data is randomly split. However, predictive models should be trained on historical data and valiadted and tested on future data. Due to these problems, validation and testing based on random sampling may provide overly optimistic results. Since random sampling is not a viable approach here, cross validation methods that rely on random samples such as k-fold cross validation is not useful either.

For predictive maintenance problems, a time-dependent spliting strategy is often a better approach to estimate performance which is done by validating and testing on examples that are later in time than the training examples. For a time-dependent split, a point in time is picked and model is trained on examples up to that point in time, and validated on the examples after that point assuming that the future data after the splitting point is not known. However, this effects the labelling of features falling into the labelling window right before the split as it is assumed that failure information is not known beyond the splitting cut-off. Due to that, those feature records can not be labeled and will not be used. This also prevents the leaking problem at the splitting point.

Validation can be performed by picking different split points and examining the performance of the models trained on different time splits. In the following, we use a splitting points to train the model and look at the performances for the other split in the evaluation section.

In [4]:
# define list of input columns for downstream modeling - note model variable was removed as string was not supported
input_features = [
'volt_rollingmean_3',
'rotate_rollingmean_3',
'pressure_rollingmean_3',
'vibration_rollingmean_3',
'volt_rollingmean_24',
'rotate_rollingmean_24',
'pressure_rollingmean_24',
'vibration_rollingmean_24',
'volt_rollingstd_3',
'rotate_rollingstd_3',
'pressure_rollingstd_3',
'vibration_rollingstd_3',
'volt_rollingstd_24',
'rotate_rollingstd_24',
'pressure_rollingstd_24',
'vibration_rollingstd_24',
'error1sum_rollingmean_24',
'error2sum_rollingmean_24',
'error3sum_rollingmean_24',
'error4sum_rollingmean_24',
'error5sum_rollingmean_24',
'comp1sum',
'comp2sum',
'comp3sum',
'comp4sum',
'age',
'model_encoded'    
]

label_var = ['label_e']
key_cols =['machineID','dt_truncated']


In [5]:
# assemble features
va = VectorAssembler(inputCols=(input_features), outputCol='features')
data = va.transform(data).select('machineID','dt_truncated','label_e','features')

In [6]:
# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(data)

In [7]:
# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="label_e", outputCol="indexedLabel").fit(data)

In [8]:
# split the data into train/test based on date
training = data.filter(data.dt_truncated > "2015-01-01").filter(data.dt_truncated < "2015-09-30")
testing = data.filter(data.dt_truncated > "2015-09-30")

print(training.count())
print(testing.count())

2174000
747000


# Classification models

In this notebook we will compare two models namely Random Forest Classifier and Decision Tree Classifier. The user can add in more models and compare each model with varying hyperparameters. 

## Random Forest Classifier

In [9]:
# train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# chain indexers and forest in a Pipeline
pipeline_rf = Pipeline(stages=[labelIndexer, featureIndexer, rf])

# train model.  This also runs the indexers.
model_rf = pipeline_rf.fit(training)

In [10]:
# make predictions.
predictions_rf = model_rf.transform(testing)
predictions_rf.groupby('indexedLabel', 'prediction').count().show()

+------------+----------+------+
|indexedLabel|prediction| count|
+------------+----------+------+
|         2.0|       0.0|  1208|
|         1.0|       1.0|  4695|
|         0.0|       1.0|   186|
|         0.0|       4.0|     4|
|         2.0|       2.0|  2422|
|         1.0|       0.0|     7|
|         4.0|       4.0|    43|
|         0.0|       0.0|734552|
|         0.0|       2.0|    60|
|         4.0|       0.0|  1680|
|         3.0|       3.0|    61|
|         3.0|       0.0|  2082|
+------------+----------+------+



In [11]:
predictionAndLabels = predictions_rf.select("indexedLabel", "prediction").rdd

In predictive maintenance, machine failures are usually rare occurrences in the lifetime of the assets compared to normal operation. This causes an imbalance in the label distribution which usually causes poor performance as algorithms tend to classify majority class examples better at the expense of minority class examples as the total misclassification error is much improved when majority class is labeled correctly. This causes low recall rates although accuracy can be high and becomes a larger problem when the cost of false alarms to the business is very high. To help with this problem, sampling techniques such as oversampling of the minority examples are usually used along with more sophisticated techniques which are not covered in this notebook.

Also, due to the class imbalance problem, it is important to look at evaluation metrics other than accuracy alone and compare those metrics to the baseline metrics which are computed when random chance is used to make predictions rather than a machine learning model. The comparison will bring out the value and benefits of using a machine learning model better.

In the following, we compute weighted precision/recall, F1 score along with the accuracy metric. 

In [12]:
# select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction")
print("Accuracy = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "accuracy"}))

Accuracy = 0.993003


In [13]:
print("Weighted Precision = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "weightedPrecision"}))

Weighted Precision = 0.992826


In [14]:
print("Weighted Recall = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "weightedRecall"}))

Weighted Recall = 0.993003


In [15]:
print("F1 = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "f1"}))

F1 = 0.990473


# Decision Tree Classifier

In [16]:
# train a DT model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# chain indexers and forest in a Pipeline
pipeline_dt = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# train model.  This also runs the indexers.
model_dt = pipeline_dt.fit(training)

In [17]:
# make predictions.
predictions_dt = model_dt.transform(testing)
predictions_dt.groupby('indexedLabel', 'prediction').count().show()

+------------+----------+------+
|indexedLabel|prediction| count|
+------------+----------+------+
|         1.0|       1.0|  4698|
|         0.0|       1.0|     4|
|         0.0|       4.0|   259|
|         2.0|       2.0|  3630|
|         1.0|       0.0|     4|
|         4.0|       4.0|  1684|
|         0.0|       0.0|734422|
|         4.0|       3.0|    28|
|         0.0|       2.0|   116|
|         4.0|       0.0|    11|
|         3.0|       3.0|  1868|
|         0.0|       3.0|     1|
|         3.0|       0.0|   275|
+------------+----------+------+



In [18]:
predictionAndLabels = predictions_dt.select("indexedLabel", "prediction").rdd

In [19]:
# select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction")
print("Accuracy = %g" % evaluator.evaluate(predictions_dt, {evaluator.metricName: "accuracy"}))

Accuracy = 0.999066


In [20]:
print("Weighted Precision = %g" % evaluator.evaluate(predictions_dt, {evaluator.metricName: "weightedPrecision"}))

Weighted Precision = 0.999105


In [21]:
print("Weighted Recall = %g" % evaluator.evaluate(predictions_dt, {evaluator.metricName: "weightedRecall"}))

Weighted Recall = 0.999066


In [22]:
print("F1 = %g" % evaluator.evaluate(predictions_dt, {evaluator.metricName: "f1"}))

F1 = 0.999066


In the next notebook Code\operationalization.ipynb Jupyter notebook we will learn how to create the functions needed to operationalize and deploy any model to get realtime predictions. 